# Section 9 — Best Practices & Deployment
Guidelines and practical snippets for organizing projects, documentation, version control, configuration, packaging, containers, and deployment.

## 9.1 Project Organization
A clean, predictable structure improves collaboration and maintainability.

```
my-python-project/
├─ README.md
├─ pyproject.toml            # or setup.cfg/setup.py
├─ requirements.txt          # or poetry.lock
├─ .env.example
├─ .gitignore
├─ src/
│  └─ mypkg/
│     ├─ __init__.py
│     ├─ core.py
│     └─ utils.py
├─ tests/
│  └─ test_core.py
├─ notebooks/
├─ data/ (raw, interim, processed)
└─ docs/
```
**Tips:** use `src/` layout, keep `data/` out of version control, and add docstrings everywhere.

## 9.2 Documentation
- **Docstrings**: write function/class/module docstrings (PEP 257).
- **Sphinx**: powerful documentation generator for Python projects.
- **MkDocs**: Markdown-first, fast and simple.

### Minimal Example Docstring

In [ ]:
def add(a: int, b: int) -> int:
    """Add two integers.

    Parameters
    ----------
    a : int
        First number.
    b : int
        Second number.

    Returns
    -------
    int
        Sum of a and b.
    """
    return a + b

help(add)

### Sphinx Quickstart (conceptual)
```bash
pip install sphinx
sphinx-quickstart docs
sphinx-apidoc -o docs/source src/mypkg
make html  # inside docs
```

## 9.3 Version Control (Git & GitHub)
- Commit early and often, with meaningful messages.
- Use feature branches and pull requests.
- Protect `main` with PR reviews and CI.

### Basic Commands

In [ ]:
git_cheats = '''
git init
git add .
git commit -m "Initial commit"
git branch -M main
git remote add origin https://github.com/<user>/<repo>.git
git push -u origin main
'''
print(git_cheats)

## 9.4 Environment Variables & Configuration
Use environment variables for secrets and configuration. Avoid hardcoding sensitive data.

### Example with `python-dotenv`

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path='.env', override=True)
API_KEY = os.getenv('API_KEY', 'missing')
print('API_KEY loaded? ', API_KEY != 'missing')

## 9.5 Packaging
Prefer **`pyproject.toml`** (PEP 517/518) with `setuptools` or `poetry`.

### Minimal `pyproject.toml` (setuptools)
```toml
[build-system]
requires = ["setuptools", "wheel"]
build-backend = "setuptools.build_meta"

[project]
name = "mypkg"
version = "0.1.0"
description = "My sample package"
requires-python = ">=3.10"
dependencies = [
  "pandas>=2.0",
]
```
### Poetry (alternative)
```bash
pipx install poetry
poetry init
poetry add pandas
poetry build
poetry publish
```

## 9.6 Docker & Containers
Containerize apps for reproducible deploys.

### Minimal `Dockerfile`
```Dockerfile
FROM python:3.11-slim
WORKDIR /app
COPY pyproject.toml .
RUN pip install --upgrade pip && pip install -e .
COPY . .
CMD ["python", "-m", "mypkg"]
```
### Build & Run
```bash
docker build -t mypkg:latest .
docker run --rm -p 8080:8080 --env-file .env mypkg:latest
```

## 9.7 Deploying Models & Apps
- **Streamlit** for quick data apps.
- **Flask/FastAPI** for REST APIs.
- Host on **Docker + cloud** (e.g., AWS, GCP, Azure, Render).

### Minimal Flask API Example

In [ ]:
from flask import Flask, jsonify, request
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    payload = request.get_json(force=True)
    # Dummy model logic
    score = sum(payload.get('features', []))
    return jsonify({'score': score})

print('Save this to app.py and run: flask run')

### Streamlit (conceptual)
```bash
pip install streamlit
streamlit hello
```
Minimal app:
```python
import streamlit as st
st.title('My Data App')
x = st.slider('Pick a number', 0, 10, 3)
st.write('You picked', x)
```

## 9.8 CI/CD (Conceptual)
- Use **GitHub Actions** to run tests, linters, and build on every PR.
- Enforce branch protections and code reviews.
- Deploy automatically on tagged releases.

### Minimal GitHub Actions Workflow (`.github/workflows/ci.yml`)
```yaml
name: CI
on: [push, pull_request]
jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-python@v5
        with:
          python-version: '3.11'
      - run: pip install -r requirements.txt
      - run: pip install pytest ruff black
      - run: ruff check .
      - run: black --check .
      - run: pytest -q
```

> **Checklist:**
- Clear repo structure (`src/`, `tests/`, `docs/`).
- Docstrings + README + change log.
- Type hints and linters (Ruff), formatter (Black).
- Secrets via env vars; `.env.example` committed.
- Reproducible builds (Docker) and CI for quality gates.